# Train model for exp9

- use Wikidata knowledge graph
- use mT5-XL as base model
- pre-train on LC-QuAD 2.0 for 32 epochs
- pre-train on QALD-9-Plus all languages for 15 epochs
- with linguistic context in pre-training and fine-tuning
- without entity knowledge
- without padding, only add a `<pad>` token after each component

__Please run all commands in the root of this repo__

__All commands are tested on 3dfed server__

## Preliminary

Install miniconda on Linux

```bash
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```

Create a conda environment "exp9"

```bash
conda create -n exp9 python=3.9
```

Enter environment
```bash
conda activate exp9
```

Install requirements

```bash
pip install -r requirements.txt
```

Install SpaCy models

```bash
pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download zh_core_web_sm
python -m spacy download en_core_web_sm
python -m spacy download fr_core_news_sm
python -m spacy download de_core_news_sm
python -m spacy download ja_core_news_sm
python -m spacy download lt_core_news_sm
python -m spacy download ru_core_news_sm
python -m spacy download es_core_news_sm
python -m spacy download uk_core_news_sm
```

Setup [wandb](https://docs.wandb.ai/guides/integrations/huggingface)


1. sign up for an account in [wandb](https://wandb.ai/)
2. create a `wandb` folder in the root of this repository
3. set TMPDIR to the path of `wandb` folder
3. login in local setting `wandb login` and follow the instruction

## Generate datasets

For pre-training:

```bash
python3 code/generate_train_csv.py \
-i datasets/lcquad2/train.json \
-o datasets/lcquad2/train.csv \
-t lcquad2 \
--linguistic_context
```

For fine-tuning:

```bash
python3 code/generate_train_csv.py \
-i datasets/qald9plus/wikidata/qald_9_plus_train_wikidata.json \
-o datasets/qald9plus/wikidata/qald_9_plus_train_wikidata.csv \
-t qald \
-kg Wikidata \
-l all \
--linguistic_context
```

## Pre-train on LC-QuAD 2.0

Since pre-training and fine-tuning commands are based on the same script `train_ds.sh`, in order to save time and avoid errors, we directly provide the code in the configured bash script. You can run the following command in your terminal.

```bash
deepspeed --include=localhost:0 --master_port 60000 code/train_new.py \
    --deepspeed deepspeed/ds_config_zero3.json \
    --model_name_or_path google/mt5-xl \
    --do_train \
    --train_file datasets/lcquad2/train.csv \
    --output_dir fine-tuned_models/exp9-pretrain \
    --num_train_epochs 32 \
    --per_device_train_batch_size=16 \
    --overwrite_output_dir \
    --save_steps 6000 \
    --save_total_limit 2 \
    --report_to wandb \
    --run_name exp9-pretrain \
    --tf32 1 \
    --fp16 0 \
    --gradient_checkpointing 1 \
    --gradient_accumulation_steps 4
```

## Fine-tune on QALD-9-Plus

```bash
deepspeed --include=localhost:0 --master_port 60000 code/train_new.py \
    --deepspeed deepspeed/ds_config_zero3.json \
    --model_name_or_path fine-tuned_models/exp9-pretrain \
    --do_train \
    --train_file datasets/qald9plus/wikidata/qald_9_plus_train_wikidata.csv \
    --output_dir fine-tuned_models/exp9-fine-tune \
    --num_train_epochs 15 \
    --per_device_train_batch_size=16 \
    --overwrite_output_dir \
    --save_steps 3000 \
    --save_total_limit 2 \
    --report_to wandb \
    --run_name exp9-fine-tune \
    --tf32 1 \
    --fp16 0 \
    --gradient_checkpointing 1 \
    --gradient_accumulation_steps 4
```

## Evaluate with GERBIL

Use `eval.sh` to generate prediction files in QALD format and evaluate them with GERBIL.
`eval.sh` is configured. 

```bash
./eval.sh
```

Prediction files are stored in `pred_files/exp9-fine-tune`.
The script uploads them to GERBIL along with the reference test file
and waits for 5 minutes for the results.
If the GERBIL experiment terminates, the results are stored in `pred_files/exp9-fine-tune/result.csv`, else, the experiment id is stored in this file. You can use the following commands to generate a csv files for results:

```bash
python3 code/gerbil_eval.py --experiment_id [experiment_id] --pred_path pred_files/exp9-fine-tune
```
